In [1]:
import numpy as np
import theano
import theano.tensor as T
from theano.sandbox.cuda.dnn import dnn_conv
from theano.tensor.nnet import conv2d, relu
from six.moves import cPickle
#data from http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import glob
from PIL import Image
from glob import glob
import scipy.signal

In [3]:
image_list = map(lambda x: x.resize((64,64)),
                 map(Image.open, glob("../data/img_align_celeba/img_align_celeba/*.jpg")))
image_array = np.array(list(map(np.asarray, image_list)))

In [4]:
plt.imshow(image_array[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def rescale(x, old=[0,255], new=[-1,1], invert = False):
    if invert:
        temp = old
        old = new
        new = temp
        
    return (x - old[0])/(old[1] - old[0]) * (new[1] -new[0]) + new[0]

In [ ]:
def deconv(X, w, subsample=(1, 1), border_mode=(0, 0), conv_mode='conv'):
    #https://github.com/Newmu/dcgan_code/lib/ops.py
    from theano.sandbox.cuda.basic_ops import (gpu_contiguous, gpu_alloc_empty)
    from theano.sandbox.cuda.dnn import GpuDnnConvDesc, GpuDnnConvGradI
    """ 
    sets up dummy convolutional forward pass and uses its grad as deconv
    currently only tested/working with same padding
    """
    img = gpu_contiguous(X)
    kerns = gpu_contiguous(w)
    desc = GpuDnnConvDesc(border_mode=border_mode, subsample=subsample,
                          conv_mode=conv_mode)(gpu_alloc_empty(img.shape[0], kerns.shape[1], img.shape[2]*subsample[0], img.shape[3]*subsample[1]).shape, kerns.shape)
    out = gpu_alloc_empty(img.shape[0], kerns.shape[1], img.shape[2]*subsample[0], img.shape[3]*subsample[1])
    d_img = GpuDnnConvGradI()(kerns, img, out, desc)
    return d_img

In [ ]:
def lrelu(x, leak=0.2):
    #https://github.com/bamos/dcgan-completion.tensorflow/blob/master/ops.py
    return(T.nnet.relu(x, alpha=leak))

In [ ]:
def batchnorm(X, g=None, b=None, u=None, s=None, a=1., e=1e-8):
    """
    batchnorm with support for not using scale and shift parameters
    as well as inference values (u and s) and partial batchnorm (via a)
    will detect and use convolutional or fully connected version
    """
    if X.ndim == 4:
        if u is not None and s is not None:
            b_u = u.dimshuffle('x', 0, 'x', 'x')
            b_s = s.dimshuffle('x', 0, 'x', 'x')
        else:
            b_u = T.mean(X, axis=[0, 2, 3]).dimshuffle('x', 0, 'x', 'x')
            b_s = T.mean(T.sqr(X - b_u), axis=[0, 2, 3]).dimshuffle('x', 0, 'x', 'x')
        if a != 1:
            b_u = (1. - a)*0. + a*b_u
            b_s = (1. - a)*1. + a*b_s
        X = (X - b_u) / T.sqrt(b_s + e)
        if g is not None and b is not None:
            X = X*g.dimshuffle('x', 0, 'x', 'x') + b.dimshuffle('x', 0, 'x', 'x')
    elif X.ndim == 2:
        if u is None and s is None:
            u = T.mean(X, axis=0)
            s = T.mean(T.sqr(X - u), axis=0)
        if a != 1:
            u = (1. - a)*0. + a*u
            s = (1. - a)*1. + a*s
        X = (X - u) / T.sqrt(s + e)
        if g is not None and b is not None:
            X = X*g + b
    else:
        raise NotImplementedError
    return X

In [ ]:
class ConvLayer(object):
    #https://github.com/mikesj-public/dcgan-autoencoder
    #Output must be an integer multiple of input, please use powers of 2
    def __init__(self, input,
                 input_size, output_size, 
                 num_input_filters, num_output_filters, 
                 W = None, filter_size = 5, 
                 activation = None,
                 rng = np.random.RandomState()):
        self.input = input
        
        is_deconv = output_size >= input_size

        #Size of 4d convolution tensor
        w_size = np.array([num_input_filters, num_output_filters, filter_size, filter_size]) \
                if is_deconv \
                else np.array([num_output_filters, num_input_filters, filter_size, filter_size])
        #Initialize weights
        if W == None:
            W_values = np.asarray(
                rng.normal(
                    scale = .02,
                    size = (w_size)
                ),
                dtype=theano.config.floatX
            )
            W = theano.shared(value=W_values, name='W', borrow=True)
        self.W = W
        
        conv_method = deconv if is_deconv else conv2d

        #Size of subsampling
        sub = output_size / input_size \
            if is_deconv else input_size / output_size
        sub = int(sub)
        #Border size
        if filter_size == 3:
            bm = 1
        else:
            bm = 2
            
        #Output
        lin_output = conv_method(input, W, subsample=(sub, sub), border_mode=(bm, bm))
        if activation is not None:
            output = activation(lin_output)
        else: output = lin_output
        self.output = output
        
        #Model parameters
        self.params = [self.W] 

In [ ]:
class FullyConnected(object):
    #http://deeplearning.net/tutorial/mlp.html
    def __init__(self, input, n_in, n_out, W = None, b = None,
                 activation = None, rng = np.random.RandomState()):
        
        self.input = input
        
        if W is None:
            W_values = np.asarray(
                rng.normal(
                    scale = .02,
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
            W = theano.shared(value=W_values, name='W', borrow=True)
        else: self.W = W

        if b is None:
            b_values = np.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)
        else: self.b = b

        self.W = W
        self.b = b
        
        # parameters of the model
        self.params = [self.W, self.b]
        
        lin_output = T.dot(input, self.W) + self.b
        self.output = (
            lin_output if activation is None
            else activation(lin_output)
        )

In [ ]:
class Generator(object):
    #https://arxiv.org/pdf/1511.06434.pdf
    def __init__(self, input, params = None, 
                 rng = np.random.RandomState(),
                 zsize = 100):
        self.input = input
        
        h_input = input
        
        h = FullyConnected(
            input=h_input,
            n_in=zsize,
            n_out=4*4*1024,
            W = params[0] if params is not None else None,
            b = params[1] if params is not None else None,
            rng=rng,
        )
        h_out = relu(batchnorm(h.output.reshape((input.shape[0],1024,4,4))))
        
        conv1 = ConvLayer(h_out, 4, 8, 1024, 512,
                          rng = rng,
                          W = params[2] if params is not None else None
                         ) 
        conv1_out = relu(batchnorm(conv1.output))
        
        conv2 = ConvLayer(conv1_out, 8, 16, 512, 256,
                          rng = rng,
                          W = params[3] if params is not None else None
                         ) 
        conv2_out = relu(batchnorm(conv2.output))
        
        conv3 = ConvLayer(conv2_out, 16, 32, 256, 128,
                          rng = rng,
                          W = params[4] if params is not None else None
                         ) 
        conv3_out = relu(batchnorm(conv3.output))
        
        conv4 = ConvLayer(conv3_out, 32, 64, 128, 3,
                          rng = rng,
                          W = params[5] if params is not None else None
                         ) 
        conv4_out = T.tanh(conv4.output)
        
        self.output = conv4_out
        self.params = h.params + conv1.params + conv2.params + \
                conv3.params + conv4.params 

In [ ]:
class Critic(object):
    #https://arxiv.org/pdf/1511.06434.pdf
    def __init__(self, input, params = None, 
                 rng = np.random.RandomState()):
        self.input = input
        
        conv1 = ConvLayer(input,64,32,3,64,
                          rng = rng,
                          W = params[0] if params is not None else None
                         ) 
        conv1_out = lrelu(conv1.output)
        
        conv2 = ConvLayer(conv1_out, 32, 16, 64, 128,
                          rng = rng,
                          W = params[1] if params is not None else None
                         ) 
        conv2_out = lrelu(batchnorm(conv2.output))
        
        conv3 = ConvLayer(conv2_out, 16, 8, 128, 256,
                          rng = rng,
                          W = params[2] if params is not None else None
                         ) 
        conv3_out = lrelu(batchnorm(conv3.output))
        
        conv4 = ConvLayer(conv3_out, 8, 4, 256, 512,
                          rng = rng,
                          W = params[3] if params is not None else None
                         ) 
        conv4_out = lrelu(batchnorm(conv4.output))
        
        h_input = conv4_out.flatten(2)
        h = FullyConnected(
            input=h_input,
            n_in=512*4*4,
            n_out=1,
            W = params[4] if params is not None else None,
            b = params[5] if params is not None else None,
            rng=rng
        )
        h_out = T.nnet.sigmoid(h.output)
        
        self.output = h.output
        self.params = conv1.params + conv2.params + \
                conv3.params + conv4.params + h.params

In [ ]:
def Adam(cost, params, lr=0.0002, b1=0.9, b2=0.999, e=1e-8, c=None):
    #https://gist.github.com/Newmu/acb738767acb4788bac3
    #Standard literature says b1=.9
    #DCGAN paper says b1 = .5
    b1 = 1-b1
    b2 = 1-b2
    updates = []
    grads = T.grad(cost, params)
    i = theano.shared(np.float32(0.))
    i_t = i + 1.
    fix1 = 1. - (1. - b1)**i_t
    fix2 = 1. - (1. - b2)**i_t
    lr_t = lr * (T.sqrt(fix2) / fix1)
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        p_t = p - (lr_t * g_t) if c is None else T.clip(p - (lr_t * g_t), -c, c)
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    updates.append((i, i_t))
    return updates

In [ ]:
def RMSprop(cost, params, lr=0.00005, rho=0.99, epsilon=1e-8, c=None):
    #https://github.com/Newmu/Theano-Tutorials
    #rho = .99 is torch default, used in WGAN paper
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        update = p - lr * g if c is None else T.clip(p - lr * g, -c, c)
        updates.append((p, update))
    return updates

In [ ]:
def gradDesc(cost, params, lr=.00005, c = None):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        update = p - lr * g if c is None else T.clip(p - lr * g, -c, c)
        updates.append((p, update))
    return updates

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    #https://github.com/alexanderkuk/log-progress
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
class DCGAN(object):
    def __init__(self,
                 genParams = None, critParams = None,
                 zsize = 100,
                 rng = np.random.RandomState(),
                 critLosses = [],
                 genLosses = []
                ):
        self.rng = rng
        self.genParams = genParams
        self.critParams = critParams
        self.zsize = zsize
        
        self.critLosses = critLosses
        self.genLosses = genLosses
        
    def train(self, X, iters = 600000,
              alpha = .00005,
              momentum = .9, m = 64, ncrit = 5,
              optimizer = "Adam",
              verbose = False, runningSave = False,
              runningFile="Running Save/DCGAN"):
        
        print("Building model...")
         # allocate symbolic variables for the data
        z = T.matrix('z')  
        x = T.tensor4('x')
        
        #Generator setup
        gen = Generator(z, zsize=self.zsize, params=self.genParams, rng=self.rng)
        self.gen = gen
        self.genParams = gen.params
        
        #Critic setup
        critTarget = Critic(x, params=self.critParams, rng=self.rng)
        self.critTarget = critTarget
        #critTrue params must be the same as critG params
        critG = Critic(gen.output, params=critTarget.params, rng=self.rng)
        self.critG = critG
        self.critParams = critTarget.params
        
        print("Building optimizers...")
        genLoss = T.mean(T.log(1-self.critG.output))
        critLoss = T.mean(T.log(self.critTarget.output)) \
                    + T.mean(T.log(1-self.critG.output))
        if optimizer == "Adam":
            genUpdates = Adam(genLoss, self.genParams, lr=alpha, b1=momentum)
            critUpdates = Adam(-critLoss, self.critParams, lr=alpha, b1=momentum)
        elif optimizer == "RMSprop":
            genUpdates = RMSprop(genLoss, self.genParams, lr=alpha, rho=momentum)
            critUpdates = RMSprop(-critLoss, self.critParams, lr=alpha, rho=momentum)
        else:
            genUpdates = gradDesc(genLoss, genParams, lr=alpha)
            critUpdates = gradDesc(-critLoss, critParams, lr=alpha)
            optimizer = "Vanilla SGD"
        print("Using " + optimizer)
        
        #Training functions
        trainGen = theano.function(
            inputs = [z],
            outputs = genLoss,
            updates = genUpdates
        )
        trainCrit = theano.function(
            inputs = [z,x],
            outputs = critLoss,
            updates = critUpdates
        )
        print("Done!")
        
        print("\nBegin training...")
        def printIter(i):
             print("Iteration %i/%i (%.2f%%)" % 
                         (
                            i+1,
                            iters,
                            100*(i+1)/iters,
                         )
                     )
                
        #Begin training
        for i in log_progress(range(iters), every = 1, name = "Iteration"):
            #Print stuff
            if i == 0 or verbose or (i+1) % 500 == 0 or iters <= 500:
                printIter(i)
            if (i+1) % 500 == 0:
                #Plot loss
                self.plot()
                #Show images
                import itertools
                k = 10
                Z = np.random.uniform(-1,1,(k*k,100))
                Z = Z.astype("float32")
                gX = self.sample(None, Z)
                fig, axes = plt.subplots(k, k, figsize=(20,20))
                for row, col in itertools.product(range(k), range(k)):
                    gx = gX[row*k+col]
                    axes[row,col].get_yaxis().set_visible(False)
                    axes[row,col].get_xaxis().set_visible(False)
                    axes[row,col].imshow(rescale((gx.transpose(1,2,0)), invert=True).astype('uint8'))
                    fig.subplots_adjust(hspace=0, wspace=0)
                plt.show()
                #Save
                if runningSave:
                    self.save(runningFile + str(i+1))
                    
            #Update parameters
            for n in range(ncrit):
                #Generate critic samples:
                Zm = self.rng.uniform(low = -1, high = 1, 
                  size=(m,self.zsize)).astype(theano.config.floatX)
                Xm = X[self.rng.randint(X.shape[0], size=m)]
                
                #Update critic
                critLoss = trainCrit(Zm, Xm)
                
            #Generate generator samples:
            Zm = self.rng.uniform(low = -1, high = 1, 
              size=(m,self.zsize)).astype(theano.config.floatX)
            
            #Update generator
            genLoss = trainGen(Zm)
            self.genLosses.append(genLoss)
            self.critLosses.append(critLoss)
        self.plot()
    
    def sample(self, n=1, Z = None):
        if Z is None:
            Z = self.rng.uniform(low = -1, high = 1, 
                      size=(n,self.zsize)).astype(theano.config.floatX)
        
        gen = Generator(Z, zsize=self.zsize, params=self.genParams, rng=self.rng)
        
        return(gen.output.eval())
    
    def plot(self, kernel_size=101):
        from matplotlib import pyplot as plt
        %matplotlib inline
        plt.figure()
        plt.plot(self.critLosses)
        plt.title("Critic loss " + str(self.critLosses[-1]))
        plt.xlabel("Generator iterations")
        plt.figure()
        plt.plot(self.genLosses)
        plt.title("Generator loss " + str(self.genLosses[-1]))
        plt.xlabel("Generator iterations")
        plt.show()
    
    def save(self, filename="DCGAN"):
        save_file = open("../models/" + filename, 'wb')  # this will overwrite current contents
        tosave = [self.genParams, self.critParams, self.rng, self.critLosses, self.genLosses]
        for param in tosave:
            # the -1 is for HIGHEST_PROTOCOL
            cPickle.dump(param, save_file, -1)  
        save_file.close()
    
    def from_file(filename="DCGAN"):
        f = open("../models/" + filename, 'rb')
        genParams = cPickle.load(f)
        critParams = cPickle.load(f)
        rng = cPickle.load(f)
        critLosses = cPickle.load(f)
        genLosses = cPickle.load(f)
        clf = DCGAN(
            genParams=genParams,
            critParams=critParams,
            rng = rng,
            critLosses=critLosses,
            genLosses=genLosses
        )
        return(clf)

In [ ]:
x = image_array.transpose(0, 3, 1, 2)

In [ ]:
x.shape

In [ ]:
#Preprocess 
x = rescale(x).astype(theano.config.floatX)

In [ ]:
#Initialize
dcgan = DCGAN(rng = np.random.RandomState(1234),
            zsize = 100
           )
#Train
dcgan.train(x, iters = 100000, m = 128, ncrit = 1,
           alpha = .0002, momentum = .5,
           optimizer = "Adam"
          )
dcgan.save()

In [ ]:
dcgan.save()

In [ ]:
import itertools
k = 10
Z = np.random.uniform(-1,1,(k*k,100))
Z = Z.astype("float32")
gX = dcgan.sample(None, Z)
fig, axes = plt.subplots(k, k, figsize=(20,20))
for row, col in itertools.product(range(k), range(k)):
    gx = gX[row*k+col]
    axes[row,col].get_yaxis().set_visible(False)
    axes[row,col].get_xaxis().set_visible(False)
    axes[row,col].imshow(rescale((gx.transpose(1,2,0)), invert=True).astype('uint8'))
    fig.subplots_adjust(hspace=0, wspace=0)

In [ ]:
import itertools
k = 10
fill1 = np.linspace(-1, 1, k)
fill2 = np.linspace(-1, 1, k)
fill = np.array(list(itertools.product(fill1,fill2)))
Z = np.zeros((k*k, 100))
#Z = np.matlib.repmat(np.random.uniform(-1,1,100), k*k, 1)
Z[:,:2] = fill
Z = Z.astype("float32")
gX = dcgan.sample(None, Z)
fig, axes = plt.subplots(k, k, figsize=(20,20))
for row, col in itertools.product(range(k), range(k)):
    gx = gX[row*k+col]
    axes[row,col].get_yaxis().set_visible(False)
    axes[row,col].get_xaxis().set_visible(False)
    axes[row,col].imshow(rescale((gx.transpose(1,2,0)), invert=True).astype('uint8'))
    fig.subplots_adjust(hspace=0, wspace=0)